In [1]:
import sys
sys.path.append('./libs')
import util_bd
import os

def obtener_especies():
    query = "SELECT m.id_especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie ORDER BY m.id_especie"
    especies = util_bd.resultados_query(query)
    return [especie[0] for especie in especies]

#limpieza
if os.path.isdir("./data"):
    shutil.rmtree("./data")
if os.path.isdir("./folds"):
    shutil.rmtree("./folds")

In [2]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed
        
def generar_fasta_especie(id_especie):
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(id_especie) + " ORDER BY cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./data/Especie" + str(id_especie) + ".lncRNA.fasta")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(id_especie) + " ORDER BY cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./data/Especie" + str(id_especie) + ".PCT.fasta")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + str(id_especie) + " ORDER BY f.cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./data/Especie" + str(id_especie) + ".CDS.fasta")

if os.path.isdir("./data"):
    shutil.rmtree("./data")
os.mkdir("./data")

%time Parallel(n_jobs=4, verbose=0)(delayed(generar_fasta_especie)(id_especie) for id_especie in obtener_especies())
print("Proceso finalizado")

CPU times: user 168 ms, sys: 48 ms, total: 216 ms
Wall time: 4min 33s
Proceso finalizado


In [3]:
#Armando folds (archivos fasta)
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil

def crear_estructura_carpetas(carpeta_fold):
    os.mkdir(carpeta_fold)
    os.mkdir(carpeta_fold + "/train")
    os.mkdir(carpeta_fold + "/test")
    os.mkdir(carpeta_fold + "/cpat")
    os.mkdir(carpeta_fold + "/features")
    os.mkdir(carpeta_fold + "/features/train")
    os.mkdir(carpeta_fold + "/features/test")

def armar_fold_final(tipo):
    with open("./folds/fold_final/train/" + tipo + ".fasta", "w+") as outfile:
        for id_especie in obtener_especies():
            with open("./data/Especie" + str(id_especie) + "." + tipo + ".fasta", "r") as infile:
                for inline in infile:
                    outfile.write(inline)
    
def armar_fold_especie(id_especie):
    crear_estructura_carpetas("./folds/fold_Especie" + str(id_especie))
    for tipo in ["lncRNA", "PCT", "CDS"]:
        with open("./folds/fold_Especie" + str(id_especie) + "/train/" + tipo + ".fasta", "w+") as outfile:
            for id_especie_sub in obtener_especies():
                with open("./data/Especie" + str(id_especie_sub) + "." + tipo + ".fasta") as infile:
                    for inline in infile:
                        if id_especie != id_especie_sub:
                            outfile.write(inline)
    for tipo in ["lncRNA", "PCT"]:
        with open("./folds/fold_Especie" + str(id_especie) + "/test/" + tipo + ".fasta", "w+") as outfile:
            with open("./data/Especie" + str(id_especie) + "." + tipo + ".fasta") as infile:
                for inline in infile:
                    outfile.write(inline)

if os.path.isdir("./folds"):
    shutil.rmtree("./folds")
os.mkdir("./folds")
crear_estructura_carpetas("./folds/fold_final")

%time Parallel(n_jobs=4, verbose=0)(delayed(armar_fold_final)(tipo) for tipo in ["lncRNA", "PCT", "CDS"])
%time Parallel(n_jobs=4, verbose=0)(delayed(armar_fold_especie)(id_especie) for id_especie in obtener_especies())
%time shutil.rmtree("./data")
    
print("Proceso finalizado")

CPU times: user 7.38 ms, sys: 4.3 ms, total: 11.7 ms
Wall time: 18.3 s
CPU times: user 86.3 ms, sys: 7.68 ms, total: 94 ms
Wall time: 6min 54s
CPU times: user 0 ns, sys: 77.4 ms, total: 77.4 ms
Wall time: 501 ms
Proceso finalizado


In [4]:
#Generando modelos CPAT de los folds
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def generar_cpat_fold(id_especie):
    archivo_lncRNA = "./folds/fold_Especie" + str(id_especie) + "/train/lncRNA.fasta"
    archivo_PCT = "./folds/fold_Especie" + str(id_especie) + "/train/PCT.fasta"
    archivo_CDS = "./folds/fold_Especie" + str(id_especie) + "/train/CDS.fasta"
    carpeta_cpat = "./folds/fold_Especie" + str(id_especie) + "/cpat"
    util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)
    
def generar_cpat_final():
    archivo_lncRNA = "./folds/fold_final/train/lncRNA.fasta"
    archivo_PCT = "./folds/fold_final/train/PCT.fasta"
    archivo_CDS = "./folds/fold_final/train/CDS.fasta"
    carpeta_cpat = "./folds/fold_final/cpat"
    util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)

def limpieza_archivos_CDS(id_especie):
    os.remove("./folds/fold_Especie" + str(id_especie) + "/train/CDS.fasta")

%time generar_cpat_final()
%time Parallel(n_jobs=4, verbose=0)(delayed(generar_cpat_fold)(id_especie) for id_especie in obtener_especies())
%time os.remove("./folds/fold_final/train/CDS.fasta")
%time Parallel(n_jobs=4, verbose=0)(delayed(limpieza_archivos_CDS)(id_especie) for id_especie in obtener_especies())
    
print("Proceso finalizado")

CPU times: user 14.8 ms, sys: 5.76 ms, total: 20.5 ms
Wall time: 8min 59s
CPU times: user 266 ms, sys: 70.8 ms, total: 337 ms
Wall time: 2h 26min 6s
CPU times: user 0 ns, sys: 4.9 ms, total: 4.9 ms
Wall time: 91.3 ms
CPU times: user 25.7 ms, sys: 25.1 ms, total: 50.8 ms
Wall time: 2.43 s
Proceso finalizado


In [5]:
#Ejecutar cpat y diamond sobre los folds
import os
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def ejecutar_cpat_fold(id_especie):
    archivo_lncRNA = "./folds/fold_Especie" + str(id_especie) + "/train/lncRNA.fasta"
    archivo_PCT = "./folds/fold_Especie" + str(id_especie) + "/train/PCT.fasta"
    carpeta_cpat = "./folds/fold_Especie" + str(id_especie) + "/cpat"
    util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace("fasta", "cpat"))
    os.remove(archivo_lncRNA.replace("fasta", "cpat") + ".dat")
    os.remove(archivo_lncRNA.replace("fasta", "cpat") + ".r")
    util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace("fasta", "cpat"))
    os.remove(archivo_PCT.replace("fasta", "cpat") + ".dat")
    os.remove(archivo_PCT.replace("fasta", "cpat") + ".r")

def ejecutar_cpat_diamond_final():
    archivo_lncRNA = "./folds/fold_final/train/lncRNA.fasta"
    archivo_PCT = "./folds/fold_final/train/PCT.fasta"
    diamond_db = "./Diamond_BD/uniprot-viridiplantae-reviewed.dmnd"
    carpeta_cpat = "./folds/fold_final/cpat"
    util_caracteristicas.ejecutar_diamond(archivo_lncRNA, diamond_db, archivo_lncRNA.replace("fasta", "dmnd"))
    util_caracteristicas.ejecutar_diamond(archivo_PCT, diamond_db, archivo_PCT.replace("fasta", "dmnd"))
    util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace("fasta", "cpat"))
    os.remove(archivo_lncRNA.replace("fasta", "cpat") + ".dat")
    os.remove(archivo_lncRNA.replace("fasta", "cpat") + ".r")
    util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace("fasta", "cpat"))
    os.remove(archivo_PCT.replace("fasta", "cpat") + ".dat")
    os.remove(archivo_PCT.replace("fasta", "cpat") + ".r")

%time ejecutar_cpat_diamond_final()
%time Parallel(n_jobs=4, verbose=0)(delayed(ejecutar_cpat_fold)(id_especie) for id_especie in obtener_especies())
    
print("Proceso finalizado")

CPU times: user 64.5 ms, sys: 34.1 ms, total: 98.6 ms
Wall time: 42min 41s
CPU times: user 241 ms, sys: 74 ms, total: 315 ms
Wall time: 2h 11min 29s
Proceso finalizado


In [6]:
#Generar archivo de features por cada fold
import os
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def generar_features_fold(id_especie):
    features_base_lncRNA = "./folds/fold_final/features/train/lncRNA.ft"
    features_base_PCT = "./folds/fold_final/features/train/PCT.ft"
    for tipo in ["train", "test"]:
        archivo_lncRNA = "./folds/fold_Especie" + str(id_especie) + "/" + tipo + "/lncRNA.fasta"
        archivo_PCT = "./folds/fold_Especie" + str(id_especie) + "/" + tipo + "/PCT.fasta"
        archivo_cpat_lncRNA = "./folds/fold_Especie" + str(id_especie) + "/train/lncRNA.cpat"
        archivo_cpat_PCT = "./folds/fold_Especie" + str(id_especie) + "/train/PCT.cpat"
        salida_lncRNA = "./folds/fold_Especie" + str(id_especie) + "/features/" + tipo + "/lncRNA.fasta"
        salida_PCT = "./folds/fold_Especie" + str(id_especie) + "/features/" + tipo + "/PCT.fasta"
        util_caracteristicas.generar_features(archivo_lncRNA, features_base_lncRNA, archivo_cpat_lncRNA, salida_lncRNA)
        util_caracteristicas.generar_features(archivo_PCT, features_base_PCT, archivo_cpat_PCT, salida_PCT)
    
def generar_features_final():
    archivo_lncRNA = "./folds/fold_final/train/lncRNA.fasta"
    archivo_PCT = "./folds/fold_final/train/PCT.fasta"
    salida_lncRNA = "./folds/fold_final/features/train/lncRNA.ft"
    salida_PCT = "./folds/fold_final/features/train/PCT.ft"
    util_caracteristicas.generar_features_base(archivo_lncRNA, archivo_lncRNA.replace("fasta", "cpat"), archivo_lncRNA.replace("fasta", "dmnd"), salida_lncRNA)
    util_caracteristicas.generar_features_base(archivo_PCT, archivo_PCT.replace("fasta", "cpat"), archivo_PCT.replace("fasta", "dmnd"), salida_PCT)

def limpieza_archivos_finales_fasta_ruta(ruta):
    shutil.rmtree(ruta + "/cpat")
    shutil.rmtree(ruta + "/test")
    shutil.rmtree(ruta + "/train")
    
def limpieza_archivos_finales_fasta(id_especie):
    limpieza_archivos_finales_fasta_ruta("./folds/fold_Especie" + str(id_especie))

%time generar_features_final()
%time Parallel(n_jobs=4, verbose=0)(delayed(generar_features_fold)(id_especie) for id_especie in obtener_especies())
%time limpieza_archivos_finales_fasta_ruta("./folds/fold_final")
%time Parallel(n_jobs=4, verbose=0)(delayed(limpieza_archivos_finales_fasta)(id_especie) for id_especie in obtener_especies())
    
print("Proceso finalizado")

CPU times: user 34.2 s, sys: 850 ms, total: 35 s
Wall time: 41.8 s


/home/jose/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 96 ms, sys: 142 ms, total: 238 ms
Wall time: 13min 45s
CPU times: user 6.23 ms, sys: 0 ns, total: 6.23 ms
Wall time: 373 ms
CPU times: user 40.3 ms, sys: 129 µs, total: 40.5 ms
Wall time: 1.77 s
Proceso finalizado
